In [198]:
from googleapiclient.discovery import build
import pandas as pd


In [199]:
api_key = ''
channel_id = 'UC_i8X3p8oZNaik8X513Zn1Q'

youtube = build('youtube', 'v3', developerKey=api_key)

### Fonction pour récupérer les infos sur la chaîne YT

In [200]:
def get_channel_stats(youtube, channel_id):
     request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
     response = request.execute()

     data = dict(Channel_name = response['items'][0]['snippet']['title'],
                 Subscribers = response['items'][0]['statistics']['subscriberCount'],
                 Total_videos = response['items'][0]['statistics']['videoCount'],
                 playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
     
     return data


In [201]:
channel_statistics = get_channel_stats(youtube, channel_id)

In [202]:
channel_data = pd.DataFrame([channel_statistics], index=['channel_statistics'])

channel_data

,Channel_name,Subscribers,Total_videos,playlist_id
channel_statistics,FilmsActu,4570000,11727,UU_i8X3p8oZNaik8X513Zn1Q


In [203]:
playlist_id = channel_data.loc[channel_data['Channel_name']=='FilmsActu', 'playlist_id'].iloc[0]
playlist_id

'UU_i8X3p8oZNaik8X513Zn1Q'

In [204]:
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId= playlist_id,
        maxResults = 50
    
    )
    response = request.execute()

    video_ids = []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId= playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    return video_ids


    # more_pages = True

    # while more_pages:
    #     if next_page_token is None:
    #         more_pages = False

    #     else:
    #         request = youtube.playlistItems().list(
    #     part="contentDetails",
    #     playlistId= playlist_id,
    #     maxResults = 50,
    #     pageToken = next_page_token
    #     )
            
    # response = request.execute()

    # for i in range(len(response['items'])):
    #     video_ids.append(response['items'][i]['contentDetails']['videoId']) 

    # next_page_token = response.get('nextPageToken')

    # return len(video_ids)


In [ ]:
video_ids = get_video_ids(youtube, playlist_id)
video_ids

### Fonction pour prendre les détails des vidéos

In [239]:
def get_video_details(youtube, video_ids):
    all_video_stats =[]

    for i in range (0, len(video_ids), 50):

        request = youtube.videos().list(
            part="snippet,statistics",
            id=','.join(video_ids[i:i+50])
        )
        
        response = request.execute()

        for video in response['items']: 
            video_stats = dict(Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               View_count = video['statistics']['viewCount'],
                               Like_count = video['statistics']['likeCount'],
                               Comment_count = video['statistics'].get('commentCount', 'N/A'),
                               Tags = video['snippet'].get('tags', 'N/A'),
                               )
            all_video_stats.append(video_stats)

    return all_video_stats
    


In [241]:
video_details = get_video_details(youtube, video_ids)

df = pd.json_normalize(video_details)

df.to_csv('video_details.csv', index=False)